In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd

from web_analytics import Schedules
from web_analytics.utils import merge_schedules, add_times_from_center, add_travel_times

In [ ]:
CENTER_POINT = (-3.70331, 40.41688)
TRAVEL_SPEED = 4.5

DISTRICTS = [
    "Centro",
    "Arganzuela",
    "Retiro",
    "Salamanca",
    "Chamartín",
    "Tetuán",
    "Chamberí",
    "Fuencarral-El Pardo",
    "Moncloa-Aravaca",
    "Latina",
    "Carabanchel",
    "Usera",
    "Puente de Vallecas",
    "Moratalaz",
    "Ciudad Lineal",
    "Hortaleza",
    "Villaverde",
    "Villa de Vallecas",
    "Vicálvaro",
    "San Blas - Canillejas",
    "Barajas",
]

In [ ]:
city_graph = ox.graph_from_place("Madrid, Spain", network_type="walk")
district_graphs_raw = {name: ox.graph_from_place(f"{name}, Madrid, Spain") for name in DISTRICTS}

In [ ]:
district_boundaries: gpd.GeoDataFrame = ox.features_from_place(
    "Madrid, Spain", {"boundary": "administrative", "admin_level": "9"}
)
district_boundaries = district_boundaries[
    (district_boundaries["admin_level"] == "9") & district_boundaries["name"].isin(DISTRICTS)
].reset_index(drop=True)

In [ ]:
add_travel_times(city_graph, TRAVEL_SPEED)

schedules = Schedules()
merge_schedules(city_graph, nx.MultiDiGraph(nx.union_all(schedules.get_all_trip_graphs().values())))

In [ ]:
center_node = ox.nearest_nodes(city_graph, CENTER_POINT[0], CENTER_POINT[1])

# sanity check
(
    city_graph.nodes[center_node]["y"] - CENTER_POINT[1],
    city_graph.nodes[center_node]["x"] - CENTER_POINT[0],
)

In [ ]:
add_times_from_center(city_graph, center_node, weight="time", dest="time_from_center")
add_times_from_center(city_graph, center_node, weight="length", dest="distance_from_center")

In [ ]:
def intersection(a: nx.Graph, b: nx.Graph):
    a_copy = a.copy()
    a_copy.remove_nodes_from(n for n in a if n not in b)
    a_copy.remove_edges_from(e for e in a.edges if e not in b.edges)
    return a_copy


district_graphs = {
    name: intersection(city_graph, graph) for name, graph in district_graphs_raw.items()
}

## Average travel times to center by district

In [ ]:
avg_travel_times = pd.DataFrame(
    {
        "District": name,
        "Time to center": sum(time for _, time in graph.nodes(data="time_from_center"))
        / len(graph),
    }
    for name, graph in district_graphs.items()
)
avg_travel_times.sort_values("Time to center", inplace=True)
avg_travel_times.reset_index(drop=True, inplace=True)

avg_travel_times

In [ ]:
avg_times_geo = district_boundaries.merge(
    right=avg_travel_times, left_on="name", right_on="District"
)
avg_times_geo.explore(column="Time to center", cmap="plasma_r", tooltip=["name", "Time to center"])

## Distance / Time ratio by district

In [ ]:
distance_time_ratios = pd.DataFrame(
    {
        "District": name,
        "Distance / Time": sum(
            data["distance_from_center"] / data["time_from_center"]
            for node, data in graph.nodes(data=True)
            if node != center_node
        )
        / len(graph),
    }
    for name, graph in district_graphs.items()
)
distance_time_ratios.sort_values("Distance / Time", inplace=True)
distance_time_ratios.reset_index(drop=True, inplace=True)

distance_time_ratios

In [ ]:
ratios_geo = district_boundaries.merge(
    right=distance_time_ratios, left_on="name", right_on="District"
)
ratios_geo.explore(column="Distance / Time", cmap="plasma", tooltip=["name", "Distance / Time"])